# Delta Practice

### File Read In

In [107]:
import pandas as pd

filepath1 = r'/Users/cartersocha/Downloads/instgramHashtagCounts.xlsx'
instaHashtagDf = pd.read_excel(filepath1)

filepath2 = r'/Users/cartersocha/Downloads/tweetCountTest.xlsx'
dailyTweetDf = pd.read_excel(filepath2)

filepath3 = r'/Users/cartersocha/Desktop/ReleaseData.xlsx'
releaseDf = pd.read_excel(filepath3, "ShowInfoEndStart")

filepath4 = r'/Users/cartersocha/Downloads/instgramAccountCounts.xlsx'
igAccountDf = pd.read_excel(filepath4)

filepath5 = r'/Users/cartersocha/Downloads/redditCountTest.xlsx'
redditSubsDf = pd.read_excel(filepath5)

filepath6 = r'/Users/cartersocha/Downloads/redditCommentData.xlsx'
redditCommentsDf = pd.read_excel(filepath6)

filepath7 = r'/Users/cartersocha/Downloads/googleTvCount.xlsx'
googleTrendsDf = pd.read_excel(filepath7)

filepath8 = r'/Users/cartersocha/Desktop/ReleaseData.xlsx'
releaseDateDf = pd.read_excel(filepath8, "ReleaseDateData")

filepath10 = r'/Users/cartersocha/Downloads/googleDataset.csv'
googleDailyData = pd.read_csv(filepath10)

### Functions

In [108]:
def DateTimeConvert(dateDf, dateColumn):
    dateDf[dateColumn] = pd.to_datetime(dateDf[dateColumn])  

    return dateDf

In [109]:
def DiffMaker(fillnaDf, valueColumn, dateColumn):
    fillnaDf.sort_values(['TvShow', dateColumn], inplace=True)

    fillnaDf['diffs'] = fillnaDf.groupby(['TvShow'])[valueColumn].transform(lambda x: x.diff()).fillna(0)

    fillnaDf.sort_index(inplace=True)

    return fillnaDf

In [110]:
def RemoveData(release,showDf):

    bigDf = pd.DataFrame()

    for i in range(len(release)):
        show = release['TvShow'][i]
        firstDate = release['Release Date'][i]
        secondDate = release['90DayDate'][i]

        smallDf = showDf[showDf['TvShow'] == show]

        newdf = smallDf[smallDf['RunDate'].between(firstDate, secondDate)]

        bigDf = bigDf.append(newdf,ignore_index=True)

    return bigDf

In [111]:
def MergeDfs(mainDf, secondDf, columnName):

    merged = pd.merge(mainDf,secondDf, how='outer', on=columnName)
    return merged

In [112]:
import numpy as np

def NegativeDiffs(diffDf,columnName):

    diffDf['zeroedDiffs'] = np.where((diffDf[columnName] < 0), 0, diffDf[columnName])

    return diffDf

In [113]:
def SumSocialMedia(sumDf,columnName):
    # check to see if this is episode count or generic summarization
    if type(columnName) != list:
        summarizedDf = sumDf.groupby(columnName, as_index=False).sum()
        summarizedDf = pd.DataFrame(summarizedDf)
    
    summarizedDf = sumDf.groupby(columnName, as_index=False).count()
    summarizedDf = pd.DataFrame(summarizedDf)
    
    return summarizedDf

### Data Transformations

##### Release Data

In [114]:
releaseDf['90DayDate'] = releaseDf['Release Date'] + pd.DateOffset(days=120)

releaseDf = DateTimeConvert(releaseDf,'90DayDate')
releaseDf = DateTimeConvert(releaseDf,'Release Date')

##### Instagram Hashtag Data

In [115]:
instaHashtagDf = DateTimeConvert(instaHashtagDf,'RunDate')

instaHashtagDf = DiffMaker(instaHashtagDf,'HashtagValue','RunDate')

instaHashDf = RemoveData(releaseDf, instaHashtagDf)
instaHashDf = NegativeDiffs(instaHashDf, 'diffs')
instaHashDf['SocialMediaSource'] = 'InstagramHashtag'
instaHashDf['SocialMediaValue'] = instaHashDf['zeroedDiffs']

#iHashtagDf = SumSocialMedia(instaHashDf, 'TvShow')

#iHashtagDf['SocialMediaValue'] = iHashtagDf['zeroedDiffs']
#iHashtagDf['SocialMediaSource'] = 'InstagramHashtag'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_11479/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Twitter Data

In [116]:
dailyTweetDf = DateTimeConvert(dailyTweetDf,'RunDate')
dailyTweetDf = RemoveData(releaseDf, dailyTweetDf)
dailyTweetDf['SocialMediaValue'] = dailyTweetDf['TweetCount'] + dailyTweetDf['RetweetCount']
dailyTweetDf['SocialMediaSource'] = 'Tweets'

#iTweetCountDf = SumSocialMedia(dailyTweetDf, 'TvShow')
#iTweetCountDf['SocialMediaSource'] = 'Tweets'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_11479/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Reddit Comment Data

In [117]:
redditCommentsDf['TvShow'] = redditCommentsDf['Show']
redditCommentsDf['RunDate'] = redditCommentsDf['runDate']
redditCommentsDf = DateTimeConvert(redditCommentsDf,'RunDate')

redCommentDf = RemoveData(releaseDf, redditCommentsDf)
redCommentDf['SocialMediaValue'] = redCommentDf['NumComments'] + redCommentDf['score']
redCommentDf['SocialMediaSource'] = 'RedditComments'

#iRedCommentDf = SumSocialMedia(redCommentDf, ['TvShow','RunDate'])
#iRedCommentDf['SocialMediaValue'] = redCommentDf['NumComments'] + redCommentDf['score']
#iRedCommentDf['SocialMediaSource'] = 'RedditComments'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_11479/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Instagram Account Data

In [119]:
igAccountDf = DateTimeConvert(igAccountDf,'RunDate')

igAccountDf = DiffMaker(igAccountDf,'IgAccountCounts','RunDate')


instaAccDf = RemoveData(releaseDf, igAccountDf)
instaAccDf = NegativeDiffs(instaAccDf, 'diffs')
instaAccDf['SocialMediaSource'] = 'InstagramAccount'
instaAccDf['SocialMediaValue'] = instaAccDf['zeroedDiffs']

#iAccountDf = SumSocialMedia(instaAccDf, 'TvShow')

#iAccountDf['SocialMediaValue'] = iAccountDf['zeroedDiffs']
#iAccountDf['SocialMediaSource'] = 'InstagramAccount'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_11479/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Reddit Subscriber Data

In [120]:
redditSubsDf = DateTimeConvert(redditSubsDf,'RunDate')

redditSubsDf = DiffMaker(redditSubsDf,'RedditSubs','RunDate')

redSubDf = RemoveData(releaseDf, redditSubsDf)
redSubDf = NegativeDiffs(redSubDf, 'diffs')
redSubDf['SocialMediaSource'] = 'RedditSubscribers'
redSubDf['SocialMediaValue'] = redSubDf['zeroedDiffs']

#iRedSubDf = SumSocialMedia(redSubDf, 'TvShow')
#iRedSubDf['SocialMediaValue'] = iRedSubDf['zeroedDiffs']
#iRedSubDf['SocialMediaSource'] = 'RedditSubscribers'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_11479/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Google Data

googleTrendsDf = DateTimeConvert(googleTrendsDf,'RunDate')

googDf = RemoveData(releaseDf, googleTrendsDf)
googDf = NegativeDiffs(googDf, 'GoogleValue')
googDf['SocialMediaSource'] = 'Google'
googDf['SocialMediaValue'] = googDf['zeroedDiffs']

In [122]:
googleDailyData = DateTimeConvert(googleDailyData,'RunDate')
googDf1 = RemoveData(releaseDf, googleDailyData)
googDf1 = NegativeDiffs(googDf1, 'GoogleValue')
googDf1['SocialMediaSource'] = 'Google'
googDf1['SocialMediaValue'] = googDf1['zeroedDiffs']

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_11479/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


,Unnamed: 0,RunDate,UnscaledData,MonthlyData,IsPartial,Scale,GoogleValue,TvShow,zeroedDiffs,SocialMediaSource,SocialMediaValue
0,5641,2022-05-13,62,62,False,0.62,38.44,theEssexSerpent,38.44,Google,38.44
1,5642,2022-05-14,100,100,False,1.00,100.00,theEssexSerpent,100.00,Google,100.00
2,5643,2022-05-15,74,74,False,0.74,54.76,theEssexSerpent,54.76,Google,54.76
3,5644,2022-05-16,39,39,False,0.39,15.21,theEssexSerpent,15.21,Google,15.21
4,5645,2022-05-17,37,37,False,0.37,13.69,theEssexSerpent,13.69,Google,13.69
...,...,...,...,...,...,...,...,...,...,...,...
5341,2729,2022-06-26,75,24,False,0.24,18.00,Outlander,18.00,Google,18.00
5342,2730,2022-06-27,67,21,False,0.21,14.07,Outlander,14.07,Google,14.07
5343,2731,2022-06-28,75,23,False,0.23,17.25,Outlander,17.25,Google,17.25
5344,2732,2022-06-29,67,21,False,0.21,14.07,Outlander,14.07,Google,14.07


##### Release Date Data

In [123]:
releaseDateDf = DateTimeConvert(releaseDateDf,'EpisodeReleaseDate')
iReleaseData = SumSocialMedia(releaseDateDf, ['TvShow','EpisodeReleaseDate'])
iReleaseData['DailyReleaseCount'] = iReleaseData['EpisodeNumber']
iReleaseData['RunDate'] = iReleaseData['EpisodeReleaseDate']

##### Join the Data

##### Time Value Datasets

In [124]:
instaAccDf = instaAccDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource','diffs']]
redSubDf = redSubDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource','diffs']]
dailyTweetDf = dailyTweetDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
instaHashDf = instaHashDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource','diffs']]
#googDf = googDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
googDf1 = googDf1[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
#redCommentDf = redCommentDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
iReleaseData = iReleaseData[['TvShow','RunDate','DailyReleaseCount']]

In [125]:
uberDf2 = pd.concat([instaAccDf,redSubDf,dailyTweetDf,instaHashDf,googDf1])

uberDf3 = MergeDfs(uberDf2, releaseDf,'TvShow')
uberDf3

uberDf3 = MergeDfs(uberDf3, iReleaseData,['TvShow','RunDate'])
uberDf3 = uberDf3[uberDf3['Ignore'] == 1]
uberDf3

,TvShow,RunDate,SocialMediaValue,SocialMediaSource,diffs,Stream,EpisodeCount,SeasonNumber,ReleaseCadence,Release Date,Ignore,ShowStatus,90DayDate,DailyReleaseCount
0,MoonKnight,2022-03-30,2481.0,InstagramAccount,2481.0,Disney,6.0,1.0,Weekly,2022-03-30,1.0,Completed,2022-07-28,1.0
1,MoonKnight,2022-03-30,134.0,RedditSubscribers,134.0,Disney,6.0,1.0,Weekly,2022-03-30,1.0,Completed,2022-07-28,1.0
2,MoonKnight,2022-03-30,165358.0,Tweets,NaN,Disney,6.0,1.0,Weekly,2022-03-30,1.0,Completed,2022-07-28,1.0
3,MoonKnight,2022-03-30,2913.0,InstagramHashtag,2913.0,Disney,6.0,1.0,Weekly,2022-03-30,1.0,Completed,2022-07-28,1.0
4,MoonKnight,2022-03-30,64.0,Google,NaN,Disney,6.0,1.0,Weekly,2022-03-30,1.0,Completed,2022-07-28,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345549,AboutLastNighthbo,2022-06-04,0.0,InstagramHashtag,0.0,HBOMax,8.0,1.0,Binge,2022-02-10,1.0,Undecided,2022-06-10,NaN
345550,AboutLastNighthbo,2022-06-05,0.0,InstagramHashtag,0.0,HBOMax,8.0,1.0,Binge,2022-02-10,1.0,Undecided,2022-06-10,NaN
345551,AboutLastNighthbo,2022-06-07,0.0,InstagramHashtag,0.0,HBOMax,8.0,1.0,Binge,2022-02-10,1.0,Undecided,2022-06-10,NaN
345552,AboutLastNighthbo,2022-06-08,0.0,InstagramHashtag,0.0,HBOMax,8.0,1.0,Binge,2022-02-10,1.0,Undecided,2022-06-10,NaN


##### Aggregation Datasets

igAccount = iAccountDf[['TvShow','SocialMediaValue','SocialMediaSource']]
redditSub = iRedSubDf[['TvShow','SocialMediaValue','SocialMediaSource']]
tweetComments = iTweetCountDf[['TvShow','SocialMediaValue','SocialMediaSource']]
igHashtag = iHashtagDf[['TvShow','SocialMediaValue','SocialMediaSource']]

##### Dataset Merging

nuberData = pd.concat([igAccount,redditSub,tweetComments,igHashtag],axis=0)
nuberData.head()

uberDf = MergeDfs(releaseDf, iHashtagDf)

uberDf = MergeDfs(uberDf, iTweetCountDf)

uberDf.head()

### Output - individuals & uber dataset

In [126]:
filepath2 = r'/Users/cartersocha/Downloads/uberDataset.csv'

uberDf3.to_csv(filepath2) 